In [68]:
import copy
import json

In [69]:
def findEntry(hits, target):
    for i in range(len(hits)):
        if hits[i]['ID'] == target:
            return i
    return None

In [70]:
def findMatches(hits, target):
    out = []
    for i in range(len(hits)):
        if hits[i]['ID'] == target:
            out.append(i)
    return out

## Create a JSON of To-Correct Spans

In [71]:
with open('../data/first_batch_simplification_rating.json') as f, open('../data/first_batch_span_fixing.json') as g:
    simp = json.load(f)
    span = sorted(json.load(g), key=lambda x: x["ID"])

In [86]:
# This just gets all sentences where the before and after the span fixing step, sentences didn't match
incorrect = []
seen = []

# iterate through the sentences in simp
for hit in simp:
    # find their corresponding ID entry in span (span is sorted so it's just an array)
    gold = span[hit['ID']]
    
    # sort both pairs of entries by system type (this should be in the same order anyways)

    # compare the sentences
    for _type in ['Deletions', 'Paraphrases', 'Splittings']:
        i = 0
        for entry in hit[_type]:
            if gold[_type][i][0] != entry[0]:
                if hit['ID'] not in [x['ID'] for x in incorrect]:
                    # record all the incorrectly saved sentences
                    incorrect.append({
                        "ID": hit['ID'],
                        "Original": gold["Original"],
                        "Deletions": [],
                        "Paraphrases": [],
                        "Splittings": []
                    })
                if gold[_type][i][0] not in [x[0] for x in incorrect[findEntry(incorrect, hit['ID'])][_type]]:
                    incorrect[findEntry(incorrect, hit['ID'])][_type].append(gold[_type][i])
            i += 1
incorrect = sorted(incorrect, key=lambda x: x["ID"])

print(len(incorrect))

3


In [73]:
# with open('../data/david_personally_fix_spans_for_these.json', 'w') as f:
#     json.dump(incorrect, f)

## Stitch in My Fixed Data

In [84]:
for fix_path in os.listdir('../mturk/david_manual_fixes_on_batch1/'):
    with open(f'../mturk/david_manual_fixes_on_batch1/{fix_path}') as f:
        fix = json.load(f)[0]
        
        # Iterates through entries in the fixed spans
        for _type in ['Deletions', 'Paraphrases', 'Splittings']:
            for entry in fix[_type]:                
                # Finds the simplifications which match the to-fix entry
                for i in findMatches(simp, fix['ID']):
                    for j in range(len(simp[i][_type])):
                        if simp[i][_type][j][1] == entry[1]:
                            
                            # Pastes the entry
                            simp[i][_type][j] = copy.deepcopy(entry)

In [89]:
with open('../data/fixed_first_batch_simplification_rating.json', 'w') as f:
    json.dump(simp, f)

## Check for Broken Spans
The interface still isn't perfect for saving spans. Determine messed up paraphrase spans

In [127]:
with open('../data/fixed_first_batch_simplification_rating.json') as f:
    fixed_simp = json.load(f)

In [128]:
for i in range(len(fixed_simp)):
    for _type in ['Deletions', 'Paraphrases', 'Splittings']:
        for entry in fixed_simp[i][_type]:
            for edit in entry[2:]:
                if edit[0] != 1 and edit[1] - edit[2] != 0:
                    print('deletion spans have non-zero differences')
                if edit[0] == 1 and edit[2] < edit[1]:
                    # print('paraphrase has its second value larger than its first value')
                    print(entry)